<a href="https://colab.research.google.com/github/MPCaiado/DLNN-Food-101/blob/main/DLNN_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.6854692200000017
GPU (s):
0.034656028999989985
GPU speedup over CPU: 77x


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

#models
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

In [6]:
!7z x /content/drive/MyDrive/EDSA/images.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/EDSA/                                      1 file, 4942431249 bytes (4714 MiB)

Extracting archive: /content/drive/MyDrive/EDSA/images.7z
--
Path = /content/drive/MyDrive/EDSA/images.7z
Type = 7z
Physical Size = 4942431249
Headers Size = 1206945
Method = LZMA2:24
Solid = +
Blocks = 3

  0%      0% 102          0% 185 - images/apple_pie/1295996.jpg                                         0% 186 - images/apple_pie/129668.jpg                                        0% 274 - images/

In [7]:
!pip install split-folders
import splitfolders
splitfolders.ratio('/content/images', output="output", seed=1337, ratio=(.7, .2, .1)) 

Copying files: 101000 files [02:26, 689.36 files/s]


**Baseline Model Development**

Model 1 - Simple Model

In [ ]:
train_datagen = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator=valid_datagen.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)



Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator.next()
#Input Layer
baseline_model_1 = Sequential()

baseline_model_1.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_1.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_1.add(Flatten())
# MLP - hidden layer:
baseline_model_1.add(Dense(2048, activation='relu'))

# MLP - output layer:
baseline_model_1.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 123008)            0         
_________________________________________________________________
dense (Dense)                (None, 2048)              251922432 
_________________________________________________________________
dense_1 (Dense)              (None, 101)               206949    
Total params: 252,132,965
Trainable params: 252,132,965
Non-trainable params: 0
_________________________________________________________________


In [ ]:
opt_1 = Adam()

baseline_model_1.compile(loss='categorical_crossentropy',
              optimizer=opt_1,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_1 = baseline_model_1.fit_generator(train_generator,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_1.save('food_baseline_model_1.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 64s 357ms/step - loss: 8.0449 - acc: 0.0153 - val_loss: 4.5351 - val_acc: 0.0281
Epoch 2/100
160/160 [==============================] - 55s 342ms/step - loss: 4.4400 - acc: 0.0391 - val_loss: 4.2706 - val_acc: 0.0547
Epoch 3/100
160/160 [==============================] - 53s 334ms/step - loss: 4.2138 - acc: 0.0716 - val_loss: 4.2398 - val_acc: 0.0750
Epoch 4/100
160/160 [==============================] - 51s 320ms/step - loss: 4.0351 - acc: 0.1058 - val_loss: 4.1751 - val_acc: 0.0922
Epoch 5/100
160/160 [==============================] - 49s 308ms/step - loss: 3.8174 - acc: 0.1535 - val_loss: 3.9599 - val_acc: 0.1375
Epoch 6/100
160/160 [==============================] - 48s 300ms/step - loss: 3.5980 - acc: 0.2102 - val_loss: 4.0499 - val_acc: 0.1078
Epoch 7/100
160/160 [==============================] - 47s 295ms/step - loss: 3.3161 - acc: 0.2763 - val_loss: 3.9403 - val_acc: 0.1219
Epoch 8/100
160/160 [===========================

Model 2 - Add Layers

In [ ]:
train_datagen_2 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_2 = ImageDataGenerator(rescale = 1./255)

test_datagen_2 = ImageDataGenerator(rescale = 1./255)

train_generator_2 = train_datagen_2.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_2 = valid_datagen_2.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)


Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_2.next()
#Input Layer
baseline_model_2 = Sequential()

baseline_model_2.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_2.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_2.add(Flatten())
# MLP - hidden layer:
baseline_model_2.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_2.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              102762496 
_________________________________________________________________
dense_3 (Dense)              (None, 101)              

In [ ]:
opt_2 = Adam()

baseline_model_2.compile(loss='categorical_crossentropy',
              optimizer=opt_2,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_2.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_2 = baseline_model_2.fit_generator(train_generator_2,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_2,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_2.save('food_baseline_model_2.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 57s 348ms/step - loss: 4.8019 - acc: 0.0146 - val_loss: 4.5286 - val_acc: 0.0219
Epoch 2/100
160/160 [==============================] - 53s 328ms/step - loss: 4.4598 - acc: 0.0397 - val_loss: 4.3371 - val_acc: 0.0547
Epoch 3/100
160/160 [==============================] - 49s 308ms/step - loss: 4.2633 - acc: 0.0606 - val_loss: 4.2741 - val_acc: 0.0703
Epoch 4/100
160/160 [==============================] - 48s 302ms/step - loss: 4.1500 - acc: 0.0781 - val_loss: 4.1532 - val_acc: 0.0859
Epoch 5/100
160/160 [==============================] - 46s 289ms/step - loss: 4.0367 - acc: 0.1029 - val_loss: 4.0991 - val_acc: 0.1031
Epoch 6/100
160/160 [==============================] - 45s 281ms/step - loss: 3.9164 - acc: 0.1204 - val_loss: 4.0028 - val_acc: 0.1016
Epoch 7/100
160/160 [==============================] - 45s 279ms/step - loss: 3.7723 - acc: 0.1444 - val_loss: 3.9216 - val_acc: 0.1219
Epoch 8/100
160/160 [===========================

Model 3 - Add Layers

In [ ]:
train_datagen_3 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_3 = ImageDataGenerator(rescale = 1./255)

test_datagen_3 = ImageDataGenerator(rescale = 1./255)

train_generator_3 = train_datagen_3.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_3 = valid_datagen_3.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_3.next()
#Input Layer
baseline_model_3 = Sequential()

baseline_model_3.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_3.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_3.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_3.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_3.add(Flatten())
# MLP - hidden layer:
baseline_model_3.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_3.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)            

In [ ]:
opt_3 = Adam()

baseline_model_3.compile(loss='categorical_crossentropy',
              optimizer=opt_3,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_3.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_3 = baseline_model_3.fit_generator(train_generator_3,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_3,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_3.save('food_baseline_model_3.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 43s 265ms/step - loss: 4.6389 - acc: 0.0104 - val_loss: 4.6077 - val_acc: 0.0141
Epoch 2/100
160/160 [==============================] - 42s 265ms/step - loss: 4.5903 - acc: 0.0173 - val_loss: 4.4821 - val_acc: 0.0250
Epoch 3/100
160/160 [==============================] - 43s 267ms/step - loss: 4.4788 - acc: 0.0262 - val_loss: 4.4175 - val_acc: 0.0375
Epoch 4/100
160/160 [==============================] - 42s 263ms/step - loss: 4.3682 - acc: 0.0426 - val_loss: 4.2752 - val_acc: 0.0656
Epoch 5/100
160/160 [==============================] - 42s 264ms/step - loss: 4.2100 - acc: 0.0677 - val_loss: 4.1563 - val_acc: 0.0750
Epoch 6/100
160/160 [==============================] - 42s 261ms/step - loss: 4.0602 - acc: 0.0872 - val_loss: 4.0526 - val_acc: 0.0953
Epoch 7/100
160/160 [==============================] - 42s 262ms/step - loss: 3.9607 - acc: 0.1048 - val_loss: 3.9026 - val_acc: 0.1219
Epoch 8/100
160/160 [===========================

Model 4 - Add more Layers

In [ ]:
train_datagen_4 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_4 = ImageDataGenerator(rescale = 1./255)

test_datagen_4 = ImageDataGenerator(rescale = 1./255)

train_generator_4 = train_datagen_4.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_4 = valid_datagen_4.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_4.next()
#Input Layer
baseline_model_4 = Sequential()

baseline_model_4.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_4.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_4.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_4.add(Flatten())
# MLP - hidden layer:
baseline_model_4.add(Dense(2048, activation='relu'))


# MLP - output layer:
baseline_model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 1024)       

In [ ]:
opt_4 = Adam()

baseline_model_4.compile(loss='categorical_crossentropy',
              optimizer=opt_4,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_4.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_4 = baseline_model_4.fit_generator(train_generator_4,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_4,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_4.save('food_baseline_model_4.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 42s 257ms/step - loss: 4.6327 - acc: 0.0119 - val_loss: 4.6153 - val_acc: 0.0172
Epoch 2/100
160/160 [==============================] - 41s 254ms/step - loss: 4.6151 - acc: 0.0108 - val_loss: 4.6161 - val_acc: 0.0125
Epoch 3/100
160/160 [==============================] - 41s 254ms/step - loss: 4.6151 - acc: 0.0129 - val_loss: 4.6169 - val_acc: 0.0078
Epoch 4/100
160/160 [==============================] - 41s 254ms/step - loss: 4.6153 - acc: 0.0087 - val_loss: 4.6143 - val_acc: 0.0094
Epoch 5/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6153 - acc: 0.0097 - val_loss: 4.6155 - val_acc: 0.0125
Epoch 6/100
160/160 [==============================] - 41s 254ms/step - loss: 4.6154 - acc: 0.0106 - val_loss: 4.6148 - val_acc: 0.0109
Epoch 7/100
160/160 [==============================] - 41s 254ms/step - loss: 4.6156 - acc: 0.0104 - val_loss: 4.6142 - val_acc: 0.0141
Epoch 8/100
160/160 [===========================

Model 5 - Dense Layers

In [ ]:
train_datagen_5 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_5 = ImageDataGenerator(rescale = 1./255)

test_datagen_5 = ImageDataGenerator(rescale = 1./255)

train_generator_5 = train_datagen_5.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_5 = valid_datagen_5.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_5.next()
#Input Layer
baseline_model_5 = Sequential()

baseline_model_5.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_5.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_5.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_5.add(Flatten())
# MLP - hidden layer:
baseline_model_5.add(Dense(2048, activation='relu'))
baseline_model_5.add(Dense(4096, activation='relu'))



# MLP - output layer:
baseline_model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_5.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 4, 4, 1024)       

In [ ]:
opt_5 = Adam()

baseline_model_5.compile(loss='categorical_crossentropy',
              optimizer=opt_5,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_5.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_5 = baseline_model_5.fit_generator(train_generator_5,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_5,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_5.save('food_baseline_model_5.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 45s 275ms/step - loss: 4.6265 - acc: 0.0131 - val_loss: 4.6158 - val_acc: 0.0078
Epoch 2/100
160/160 [==============================] - 44s 276ms/step - loss: 4.6156 - acc: 0.0092 - val_loss: 4.6159 - val_acc: 0.0141
Epoch 3/100
160/160 [==============================] - 44s 277ms/step - loss: 4.6153 - acc: 0.0102 - val_loss: 4.6147 - val_acc: 0.0125
Epoch 4/100
160/160 [==============================] - 43s 270ms/step - loss: 4.6153 - acc: 0.0112 - val_loss: 4.6163 - val_acc: 0.0047
Epoch 5/100
160/160 [==============================] - 43s 271ms/step - loss: 4.6156 - acc: 0.0099 - val_loss: 4.6140 - val_acc: 0.0109
Epoch 6/100
160/160 [==============================] - 44s 275ms/step - loss: 4.6154 - acc: 0.0113 - val_loss: 4.6160 - val_acc: 0.0109
Epoch 7/100
160/160 [==============================] - 42s 265ms/step - loss: 4.6148 - acc: 0.0093 - val_loss: 4.6142 - val_acc: 0.0109
Epoch 8/100
160/160 [===========================

Model 6 - Dense Layers

In [ ]:
train_datagen_6 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_6 = ImageDataGenerator(rescale = 1./255)

test_datagen_6 = ImageDataGenerator(rescale = 1./255)

train_generator_6 = train_datagen_6.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_6 = valid_datagen_6.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_6.next()
#Input Layer
baseline_model_6 = Sequential()

baseline_model_6.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_6.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_6.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_6.add(Flatten())
# MLP - hidden layer:
baseline_model_6.add(Dense(2048, activation='relu'))
baseline_model_6.add(Dense(4096, activation='relu'))
baseline_model_6.add(Dense(4096, activation='relu'))



# MLP - output layer:
baseline_model_6.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_6.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 4, 4, 1024)       

In [ ]:
opt_6 = Adam()

baseline_model_6.compile(loss='categorical_crossentropy',
              optimizer=opt_6,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_6.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_6 = baseline_model_6.fit_generator(train_generator_6,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_6,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_6.save('food_baseline_model_6.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 44s 270ms/step - loss: 4.6318 - acc: 0.0090 - val_loss: 4.6157 - val_acc: 0.0078
Epoch 2/100
160/160 [==============================] - 43s 268ms/step - loss: 4.6152 - acc: 0.0072 - val_loss: 4.6154 - val_acc: 0.0125
Epoch 3/100
160/160 [==============================] - 43s 270ms/step - loss: 4.6150 - acc: 0.0129 - val_loss: 4.6157 - val_acc: 0.0094
Epoch 4/100
160/160 [==============================] - 43s 271ms/step - loss: 4.6157 - acc: 0.0103 - val_loss: 4.6157 - val_acc: 0.0047
Epoch 5/100
160/160 [==============================] - 44s 272ms/step - loss: 4.6150 - acc: 0.0100 - val_loss: 4.6148 - val_acc: 0.0109
Epoch 6/100
160/160 [==============================] - 44s 272ms/step - loss: 4.6153 - acc: 0.0116 - val_loss: 4.6151 - val_acc: 0.0109
Epoch 7/100
160/160 [==============================] - 44s 274ms/step - loss: 4.6158 - acc: 0.0096 - val_loss: 4.6157 - val_acc: 0.0109
Epoch 8/100
160/160 [===========================

Model 7 - Add Dropout

In [ ]:
train_datagen_7 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_7 = ImageDataGenerator(rescale = 1./255)

test_datagen_7 = ImageDataGenerator(rescale = 1./255)

train_generator_7 = train_datagen_7.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_7 = valid_datagen_7.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_7.next()
#Input Layer
baseline_model_7 = Sequential()

baseline_model_7.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_7.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_7.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_7.add(Flatten())
# MLP - hidden layer:
baseline_model_7.add(Dense(2048, activation='relu'))
baseline_model_7.add(Dropout(0.2))



# MLP - output layer:
baseline_model_7.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_7.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 4, 4, 1024)       

In [ ]:
opt_7 = Adam()

baseline_model_7.compile(loss='categorical_crossentropy',
              optimizer=opt_7,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_7.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_7 = baseline_model_7.fit_generator(train_generator_7,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_7,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_7.save('food_baseline_model_7.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6210 - acc: 0.0100 - val_loss: 4.5640 - val_acc: 0.0094
Epoch 2/100
160/160 [==============================] - 41s 254ms/step - loss: 4.5604 - acc: 0.0160 - val_loss: 4.4878 - val_acc: 0.0188
Epoch 3/100
160/160 [==============================] - 41s 254ms/step - loss: 4.4632 - acc: 0.0317 - val_loss: 4.4009 - val_acc: 0.0344
Epoch 4/100
160/160 [==============================] - 41s 254ms/step - loss: 4.3441 - acc: 0.0507 - val_loss: 4.2987 - val_acc: 0.0516
Epoch 5/100
160/160 [==============================] - 41s 254ms/step - loss: 4.2673 - acc: 0.0540 - val_loss: 4.1766 - val_acc: 0.0844
Epoch 6/100
160/160 [==============================] - 41s 254ms/step - loss: 4.1471 - acc: 0.0758 - val_loss: 4.0332 - val_acc: 0.0938
Epoch 7/100
160/160 [==============================] - 41s 256ms/step - loss: 4.0421 - acc: 0.0928 - val_loss: 3.9713 - val_acc: 0.0984
Epoch 8/100
160/160 [===========================

Model 8 - Increase Dropout Rate

In [ ]:
train_datagen_8 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_8 = ImageDataGenerator(rescale = 1./255)

test_datagen_8 = ImageDataGenerator(rescale = 1./255)

train_generator_8 = train_datagen_8.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_8 = valid_datagen_8.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_8.next()
#Input Layer
baseline_model_8 = Sequential()

baseline_model_8.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_8.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_8.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_8.add(Flatten())
# MLP - hidden layer:
baseline_model_8.add(Dense(2048, activation='relu'))
baseline_model_8.add(Dropout(0.3))


# MLP - output layer:
baseline_model_8.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_8.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 4, 4, 1024)       

In [ ]:
opt_8 = Adam()

baseline_model_8.compile(loss='categorical_crossentropy',
              optimizer=opt_8,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_8.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_8 = baseline_model_8.fit_generator(train_generator_8,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_8,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_8.save('food_baseline_model_8.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 45s 280ms/step - loss: 4.6185 - acc: 0.0107 - val_loss: 4.5850 - val_acc: 0.0219
Epoch 2/100
160/160 [==============================] - 44s 278ms/step - loss: 4.5559 - acc: 0.0165 - val_loss: 4.4840 - val_acc: 0.0188
Epoch 3/100
160/160 [==============================] - 45s 279ms/step - loss: 4.4592 - acc: 0.0306 - val_loss: 4.4657 - val_acc: 0.0437
Epoch 4/100
160/160 [==============================] - 45s 283ms/step - loss: 4.3732 - acc: 0.0482 - val_loss: 4.2955 - val_acc: 0.0469
Epoch 5/100
160/160 [==============================] - 45s 278ms/step - loss: 4.2703 - acc: 0.0575 - val_loss: 4.2132 - val_acc: 0.0625
Epoch 6/100
160/160 [==============================] - 45s 280ms/step - loss: 4.1734 - acc: 0.0651 - val_loss: 4.1190 - val_acc: 0.0766
Epoch 7/100
160/160 [==============================] - 45s 280ms/step - loss: 4.1049 - acc: 0.0822 - val_loss: 3.9930 - val_acc: 0.0938
Epoch 8/100
160/160 [===========================

Model 9 - Add Kernel Constraint

In [ ]:
train_datagen_9 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_9 = ImageDataGenerator(rescale = 1./255)

test_datagen_9 = ImageDataGenerator(rescale = 1./255)

train_generator_9 = train_datagen_9.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_9 = valid_datagen_9.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_9.next()
#Input Layer
baseline_model_9 = Sequential()

baseline_model_9.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_9.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_9.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_9.add(Flatten())
# MLP - hidden layer:
baseline_model_9.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_9.add(Dropout(0.3))



# MLP - output layer:
baseline_model_9.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_9.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 4, 4, 1024)      

In [ ]:
opt_9 = Adam()

baseline_model_9.compile(loss='categorical_crossentropy',
              optimizer=opt_9,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_9.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_9 = baseline_model_9.fit_generator(train_generator_9,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_9,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_9.save('food_baseline_model_9.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 44s 269ms/step - loss: 4.6308 - acc: 0.0099 - val_loss: 4.6145 - val_acc: 0.0141
Epoch 2/100
160/160 [==============================] - 42s 265ms/step - loss: 4.6121 - acc: 0.0104 - val_loss: 4.6137 - val_acc: 0.0156
Epoch 3/100
160/160 [==============================] - 42s 265ms/step - loss: 4.6108 - acc: 0.0104 - val_loss: 4.5593 - val_acc: 0.0141
Epoch 4/100
160/160 [==============================] - 42s 263ms/step - loss: 4.5272 - acc: 0.0201 - val_loss: 4.4549 - val_acc: 0.0250
Epoch 5/100
160/160 [==============================] - 42s 264ms/step - loss: 4.4270 - acc: 0.0336 - val_loss: 4.3558 - val_acc: 0.0516
Epoch 6/100
160/160 [==============================] - 42s 262ms/step - loss: 4.3067 - acc: 0.0530 - val_loss: 4.2446 - val_acc: 0.0609
Epoch 7/100
160/160 [==============================] - 42s 265ms/step - loss: 4.2047 - acc: 0.0592 - val_loss: 4.1455 - val_acc: 0.0609
Epoch 8/100
160/160 [===========================

Model 10 - Add learning rate 

In [ ]:
train_datagen_10 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_10 = ImageDataGenerator(rescale = 1./255)

test_datagen_10 = ImageDataGenerator(rescale = 1./255)

train_generator_10 = train_datagen_10.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_10 = valid_datagen_10.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_10.next()
#Input Layer
baseline_model_10 = Sequential()

baseline_model_10.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_10.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_10.add(Flatten())
# MLP - hidden layer:
baseline_model_10.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_10.add(Dropout(0.3))



# MLP - output layer:
baseline_model_10.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_10.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 4, 4, 1024)      

In [ ]:
opt_10 = Adam(lr=0.001)

baseline_model_10.compile(loss='categorical_crossentropy',
              optimizer=opt_10,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_10.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_10 = baseline_model_10.fit_generator(train_generator_10,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_10,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_10.save('food_baseline_model_10.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 42s 261ms/step - loss: 4.6210 - acc: 0.0099 - val_loss: 4.6142 - val_acc: 0.0156
Epoch 2/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6138 - acc: 0.0109 - val_loss: 4.6176 - val_acc: 0.0063
Epoch 3/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6160 - acc: 0.0078 - val_loss: 4.6138 - val_acc: 0.0063
Epoch 4/100
160/160 [==============================] - 41s 257ms/step - loss: 4.6164 - acc: 0.0109 - val_loss: 4.6159 - val_acc: 0.0094
Epoch 5/100
160/160 [==============================] - 41s 258ms/step - loss: 4.6153 - acc: 0.0108 - val_loss: 4.6158 - val_acc: 0.0141
Epoch 6/100
160/160 [==============================] - 41s 256ms/step - loss: 4.6158 - acc: 0.0134 - val_loss: 4.6161 - val_acc: 0.0063
Epoch 7/100
160/160 [==============================] - 41s 255ms/step - loss: 4.6161 - acc: 0.0100 - val_loss: 4.5864 - val_acc: 0.0125
Epoch 8/100
160/160 [===========================

Model 11 - Change optimizer to RMSprop

In [ ]:
train_datagen_11 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_11 = ImageDataGenerator(rescale = 1./255)

test_datagen_11 = ImageDataGenerator(rescale = 1./255)

train_generator_11 = train_datagen_11.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_11 = valid_datagen_11.flow_from_directory(directory="/content/output/val",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_11.next()
#Input Layer
baseline_model_11 = Sequential()

baseline_model_11.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_11.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_11.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_11.add(Flatten())
# MLP - hidden layer:
baseline_model_11.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_11.add(Dropout(0.3))



# MLP - output layer:
baseline_model_11.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_11.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 4, 4, 1024)      

In [ ]:
opt_11 = RMSprop(lr=0.001)

baseline_model_11.compile(loss='categorical_crossentropy',
              optimizer=opt_11,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_11.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_11 = baseline_model_11.fit_generator(train_generator_11,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_11,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_11.save('food_baseline_model_11.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 43s 264ms/step - loss: 6.4204 - acc: 0.0089 - val_loss: 4.5830 - val_acc: 0.0188
Epoch 2/100
160/160 [==============================] - 42s 261ms/step - loss: 4.5861 - acc: 0.0179 - val_loss: 4.5479 - val_acc: 0.0172
Epoch 3/100
160/160 [==============================] - 42s 264ms/step - loss: 4.4859 - acc: 0.0283 - val_loss: 4.4569 - val_acc: 0.0375
Epoch 4/100
160/160 [==============================] - 42s 263ms/step - loss: 4.3869 - acc: 0.0374 - val_loss: 4.2968 - val_acc: 0.0609
Epoch 5/100
160/160 [==============================] - 42s 263ms/step - loss: 4.2858 - acc: 0.0585 - val_loss: 4.2977 - val_acc: 0.0609
Epoch 6/100
160/160 [==============================] - 42s 261ms/step - loss: 4.2006 - acc: 0.0717 - val_loss: 4.0878 - val_acc: 0.0906
Epoch 7/100
160/160 [==============================] - 42s 260ms/step - loss: 4.0453 - acc: 0.0883 - val_loss: 4.1260 - val_acc: 0.0750
Epoch 8/100
160/160 [===========================

Model 12 - Increase Batch Size + Steps per epoch

In [ ]:
train_datagen_12 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_12 = ImageDataGenerator(rescale = 1./255)

test_datagen_12 = ImageDataGenerator(rescale = 1./255)

train_generator_12 = train_datagen_12.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_12 = valid_datagen_12.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_12.next()
#Input Layer
baseline_model_12 = Sequential()

baseline_model_12.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_12.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_12.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_12.add(Flatten())
# MLP - hidden layer:
baseline_model_12.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_12.add(Dropout(0.3))



# MLP - output layer:
baseline_model_12.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_12.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)        4

In [ ]:
opt_12 = RMSprop(lr=0.001)

baseline_model_12.compile(loss='categorical_crossentropy',
              optimizer=opt_12,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_12.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_12 = baseline_model_12.fit_generator(train_generator_12,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_12,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_12.save('food_baseline_model_12.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
320/320 [==============================] - 213s 640ms/step - loss: 5.3678 - acc: 0.0152 - val_loss: 4.4189 - val_acc: 0.0525
Epoch 2/100
320/320 [==============================] - 178s 556ms/step - loss: 4.2862 - acc: 0.0580 - val_loss: 4.0941 - val_acc: 0.0886
Epoch 3/100
320/320 [==============================] - 170s 530ms/step - loss: 3.9380 - acc: 0.1135 - val_loss: 3.7706 - val_acc: 0.1348
Epoch 4/100
320/320 [==============================] - 165s 514ms/step - loss: 3.6328 - acc: 0.1608 - val_loss: 3.5783 - val_acc: 0.1772
Epoch 5/100
320/320 [==============================] - 163s 508ms/step - loss: 3.3493 - acc: 0.2126 - val_loss: 3.3669 - val_acc: 0.2161
Epoch 6/100
320/320 [==============================] - 162s 505ms/step - loss: 3.1299 - acc: 0.2544 - val_loss: 3.3197 - val_acc: 0.2354
Epoch 7/100
320/320 [==============================] - 161s 504ms/step - loss: 2.8967 - acc: 0.2943 - val_loss: 3.1217 - val_acc: 0.2598
Epoch 8/100
320/320 [====================

Model 13 - Change padding (Worse Results)

In [ ]:
train_datagen_13 = ImageDataGenerator(
                                  rescale = 1./255.)

valid_datagen_13 = ImageDataGenerator(rescale = 1./255)

test_datagen_13 = ImageDataGenerator(rescale = 1./255)

train_generator_13 = train_datagen_13.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_13 = valid_datagen_13.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_13.next()
#Input Layer
baseline_model_13 = Sequential()

baseline_model_13.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_13.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='same'))
baseline_model_13.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_13.add(Flatten())
# MLP - hidden layer:
baseline_model_13.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_13.add(Dropout(0.3))



# MLP - output layer:
baseline_model_13.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_13.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 128)       3584      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 512)       1180160   
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 1024)       

In [ ]:
opt_13 = RMSprop(lr=0.001)

baseline_model_13.compile(loss='categorical_crossentropy',
              optimizer=opt_13,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_13.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_13 = baseline_model_13.fit_generator(train_generator_13,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_13,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_13.save('food_baseline_model_13.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
320/320 [==============================] - 165s 509ms/step - loss: 7.5279 - acc: 0.0105 - val_loss: 4.4025 - val_acc: 0.0383
Epoch 2/100
320/320 [==============================] - 163s 508ms/step - loss: 4.3560 - acc: 0.0465 - val_loss: 3.9914 - val_acc: 0.1052
Epoch 3/100
320/320 [==============================] - 162s 507ms/step - loss: 3.9611 - acc: 0.1086 - val_loss: 3.7168 - val_acc: 0.1479
Epoch 4/100
320/320 [==============================] - 163s 508ms/step - loss: 3.5958 - acc: 0.1679 - val_loss: 3.5097 - val_acc: 0.1787
Epoch 5/100
320/320 [==============================] - 163s 508ms/step - loss: 3.3272 - acc: 0.2108 - val_loss: 3.3261 - val_acc: 0.2178
Epoch 6/100
320/320 [==============================] - 162s 507ms/step - loss: 3.0465 - acc: 0.2657 - val_loss: 3.2191 - val_acc: 0.2375
Epoch 7/100
320/320 [==============================] - 163s 508ms/step - loss: 2.8057 - acc: 0.3200 - val_loss: 3.0634 - val_acc: 0.2708
Epoch 8/100
320/320 [====================

Model 14 - Include Data Augmentation techniques

In [ ]:
train_datagen_14 = ImageDataGenerator(samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.)

valid_datagen_14 = ImageDataGenerator(rescale = 1./255)

test_datagen_14 = ImageDataGenerator(rescale = 1./255)

train_generator_14 = train_datagen_14.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_14 = valid_datagen_14.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
X_train, y_train = train_generator_14.next()
#Input Layer
baseline_model_14 = Sequential()

baseline_model_14.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_14.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_14.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_14.add(Flatten())
# MLP - hidden layer:
baseline_model_14.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_14.add(Dropout(0.3))



# MLP - output layer:
baseline_model_14.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_14.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 1024)       

In [ ]:
opt_14 = RMSprop(lr=0.001)

baseline_model_14.compile(loss='categorical_crossentropy',
              optimizer=opt_14,
              metrics=['acc'])

In [18]:
X_val, y_val = valid_generator_14.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_14 = baseline_model_14.fit_generator(train_generator_14,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_14,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_14.save('food_baseline_model_14.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
320/320 [==============================] - 165s 516ms/step - loss: 1.3322 - acc: 0.6498 - val_loss: 6.2206 - val_acc: 0.0151
Epoch 2/100
320/320 [==============================] - 164s 514ms/step - loss: 1.1822 - acc: 0.6845 - val_loss: 9.2787 - val_acc: 0.0137
Epoch 3/100
320/320 [==============================] - 165s 515ms/step - loss: 1.0812 - acc: 0.7125 - val_loss: 9.8071 - val_acc: 0.0122
Epoch 4/100
320/320 [==============================] - 165s 514ms/step - loss: 1.0017 - acc: 0.7325 - val_loss: 12.6672 - val_acc: 0.0154
Epoch 5/100
320/320 [==============================] - 165s 516ms/step - loss: 0.9470 - acc: 0.7446 - val_loss: 8.1167 - val_acc: 0.0098
Epoch 6/100
320/320 [==============================] - 165s 515ms/step - loss: 0.9068 - acc: 0.7589 - val_loss: 10.8538 - val_acc: 0.0159
Epoch 7/100
320/320 [==============================] - 165s 516ms/step - loss: 0.8851 - acc: 0.7639 - val_loss: 9.7298 - val_acc: 0.0142
Epoch 8/100
320/320 [==================

Model 15 - More Data Augmentation techniques


In [8]:
train_datagen_15 = ImageDataGenerator(samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

valid_datagen_15 = ImageDataGenerator(rescale = 1./255)

test_datagen_15 = ImageDataGenerator(rescale = 1./255)

train_generator_15 = train_datagen_15.flow_from_directory(directory="/content/output/train",
                                                    subset="training",
                                                    batch_size=128,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator_15 = valid_datagen_15.flow_from_directory(directory="/content/output/val",
                                                  batch_size=128,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [9]:
X_train, y_train = train_generator_15.next()
#Input Layer
baseline_model_15 = Sequential()

baseline_model_15.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model_15.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model_15.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# MLP - input layer:
baseline_model_15.add(Flatten())
# MLP - hidden layer:
baseline_model_15.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model_15.add(Dropout(0.3))



# MLP - output layer:
baseline_model_15.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model_15.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 1024)        4

In [10]:
opt_15 = RMSprop(lr=0.001)

baseline_model_15.compile(loss='categorical_crossentropy',
              optimizer=opt_15,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator_15.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model_15 = baseline_model_15.fit_generator(train_generator_15,
                              steps_per_epoch=320, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator_15,
                              validation_steps=32, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model_15.save('food_baseline_model_15.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
320/320 [==============================] - 3164s 10s/step - loss: 5.6386 - acc: 0.0166 - val_loss: 4.8094 - val_acc: 0.0107
Epoch 2/100
320/320 [==============================] - 3153s 10s/step - loss: 4.2988 - acc: 0.0539 - val_loss: 5.1377 - val_acc: 0.0093
Epoch 3/100
320/320 [==============================] - 3146s 10s/step - loss: 4.0046 - acc: 0.0980 - val_loss: 6.1163 - val_acc: 0.0112
Epoch 4/100
320/320 [==============================] - 3147s 10s/step - loss: 3.8144 - acc: 0.1286 - val_loss: 5.2724 - val_acc: 0.0188
Epoch 5/100
233/320 [====================>.........] - ETA: 13:55 - loss: 3.6409 - acc: 0.1631

In [ ]:
train_datagen = ImageDataGenerator(
                                  samplewise_center = True,
                                  samplewise_std_normalization = True,
                                  rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(directory="/output/train/",
                                                    subset="training",
                                                    batch_size=64,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(64,64),
                                                    seed=42)



valid_generator=test_datagen.flow_from_directory(directory="/output/val/",
                                                  subset="validation",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)


teste=test_datagen.flow_from_directory(directory="/output/test",
                                                  subset="test",
                                                  batch_size=64,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(64,64),
                                                  seed=42)


FileNotFoundError: ignored

In [ ]:


X_train, y_train = train_generator.next()
#Input Layer
baseline_model = Sequential()

baseline_model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid', input_shape=X_train.shape[1:]))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

baseline_model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


baseline_model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


baseline_model.add(Conv2D(filters=1024, kernel_size=(3, 3), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))


# MLP - input layer:
baseline_model.add(Flatten())
# MLP - hidden layer:
baseline_model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(2048, activation='relu',kernel_constraint=maxnorm(3)))
baseline_model.add(Dropout(0.2))

# MLP - output layer:
baseline_model.add(Dense(y_train.shape[1], activation='softmax'))

# Check built model:
baseline_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_110 (Conv2D)          (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 29, 29, 256)       295168    
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 12, 12, 512)       1180160   
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 6, 6, 512)         0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 4, 4, 1024)      

In [ ]:
opt = Adam(lr=0.001)

baseline_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

In [ ]:
X_val, y_val = valid_generator.next()
early_stop = EarlyStopping(patience=30, monitor='val_loss', mode='min', restore_best_weights=True)


history_baseline_model = baseline_model_1.fit_generator(train_generator,
                              steps_per_epoch=160, #steps_per_epoch,
                              epochs=100,
                              validation_data=valid_generator,
                              validation_steps=10, #validation_steps,
                              callbacks=[early_stop],
                              shuffle= True
                             )

baseline_model.save('food_baseline_model_1.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
160/160 [==============================] - 54s 336ms/step - loss: 4.2930 - acc: 0.0545 - val_loss: 4.3365 - val_acc: 0.0547
Epoch 2/100
160/160 [==============================] - 54s 337ms/step - loss: 4.2158 - acc: 0.0665 - val_loss: 4.3475 - val_acc: 0.0453
Epoch 3/100
160/160 [==============================] - 54s 336ms/step - loss: 4.1883 - acc: 0.0685 - val_loss: 4.1274 - val_acc: 0.0656
Epoch 4/100
160/160 [==============================] - 54s 338ms/step - loss: 4.1421 - acc: 0.0763 - val_loss: 4.0769 - val_acc: 0.0734
Epoch 5/100
160/160 [==============================] - 54s 337ms/step - loss: 4.1097 - acc: 0.0757 - val_loss: 4.1044 - val_acc: 0.0812
Epoch 6/100
160/160 [==============================] - 54s 336ms/step - loss: 4.0798 - acc: 0.0794 - val_loss: 4.1218 - val_acc: 0.0734
Epoch 7/100
160/160 [==============================] - 54s 337ms/step - loss: 4.0663 - acc: 0.0854 - val_loss: 4.0138 - val_acc: 0.1000
Epoch 8/100
160/160 [===========================

Evaluate Results

In [ ]:
incnet = InceptionV3(weights='imagenet', include_top=False, input_tensor=layers.Input(shape=(299, 299, 3)))
x = incnet.output
x = layers.AveragePooling2D(pool_size=(8, 8))(x)
x = layers.Dropout(.2)(x)
x = layers.Flatten()(x)
output = layers.Dense(101, activation='softmax', kernel_regularizer=regularizers.l2(.0005))(x)

model = models.Model(inputs=incnet.input, outputs=output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
gc.collect()

1005

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4)
checkpoint_callback = ModelCheckpoint('InceptionNet.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
history = model.fit_generator(train_generator,
                            validation_data=valid_generator,
                            epochs=10,
                            workers=0,
                            use_multiprocessing=False, 
                            callbacks=[early_stopping_callback, checkpoint_callback])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1263/1263 [==============================] - 1739s 1s/step - loss: 2.8844 - accuracy: 0.3382 - val_loss: 2.6619 - val_accuracy: 0.3891

Epoch 00001: val_loss improved from inf to 2.66189, saving model to InceptionNet.h5
Epoch 2/10
1263/1263 [==============================] - 1675s 1s/step - loss: 1.4509 - accuracy: 0.6439 - val_loss: 2.6808 - val_accuracy: 0.4057

Epoch 00002: val_loss did not improve from 2.66189
Epoch 3/10
1263/1263 [==============================] - 1638s 1s/step - loss: 1.1333 - accuracy: 0.7240 - val_loss: 1.8525 - val_accuracy: 0.5699

Epoch 00003: val_loss improved from 2.66189 to 1.85251, saving model to InceptionNet.h5
Epoch 4/10
1263/1263 [==============================] - 1615s 1s/step - loss: 0.9297 - accuracy: 0.7748 - val_loss: 2.3600 - val_accuracy: 0.4794

Epoch 00004: val_loss did not improve from 1.85251
Epoch 5/10
1263/1263 [==============================] - 1600s 1s/step - loss: 0.7756 - accuracy: 0.8156 - val_loss: 1.8512 - val_accuracy:

In [ ]:
train_datagen = ImageDataGenerator(#data_format='channels_first',
                                  validation_split=0.2,
                                  samplewise_center = True,
                                  samplewise_std_normalization = True,rescale = 1.0/255.)

train_datageno = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

train_generator = train_datagen.flow_from_directory(directory="/content/images",
                                                    subset="training",
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(224,224),
                                                    seed=42)

valid_generator=train_datagen.flow_from_directory(directory="/content/images",
                                                  subset="validation",
                                                  batch_size=20,
                                                  shuffle=True,
                                                  class_mode="categorical",
                                                  target_size=(224,224),
                                                  seed=42)



Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
train_datageno.fit(train_generator)

In [ ]:
train_generator

valid_generator

In [ ]:
#IMAGE AUGMENTATION

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
#TRAINING

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(directory="/content/output/train", batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(directory="/content/output/val",  batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#NOVO PROJETO

In [ ]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

In [ ]:
# Helper method to create train_mini and test_mini data samples
def dataset_mini(food_list, src, dest):
  if os.path.exists(dest):
    rmtree(dest) # removing dataset_mini(if it already exists) folders so that we will have only the classes that we want
  os.makedirs(dest)
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [ ]:
food_list = ['apple_pie','pizza','omelette']
src_train = '/content/output/train'
dest_train = '/content/output/train_mini'
src_test = '/content/output/val'
dest_test = '/content/output/val_mini'

In [ ]:
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)

Creating train data folder with new classes
Copying images into apple_pie
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in train folder")

!find /content/output/train_mini -type d -or -type f -printf '.' | wc -c



Total number of samples in train folder
2400


In [ ]:


print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)



Creating test data folder with new classes
Copying images into apple_pie
Copying images into pizza
Copying images into omelette


In [ ]:
print("Total number of samples in test folder")
!find /content/output/val_mini -type d -or -type f -printf '.' | wc -c



Total number of samples in test folder
600


In [ ]:
K.clear_session()
n_classes = 3
img_width, img_height = 299, 299
train_data_dir = '/content/output/train_mini'
validation_data_dir = '/content/output/val_mini'
nb_train_samples = 2400 #75750
nb_validation_samples = 600 #25250
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(3,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_3class.log')

history = model.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model.save('model_trained_3class.hdf5')

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
87916544/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
150/150 [==============================] - 74s 442ms/step - loss: 1.1282 - accuracy: 0.4067 - val_loss: 0.8111 - val_accuracy: 0.6959

Epoch 00001: val_loss improved from inf to 0.81108, saving model to best_model_3class.hdf5
Epoch 2/30
150/150 [==============================] - 65s 432ms/step - loss: 0.8120 - accuracy: 0.6959 - val_loss: 0.6171 - val_accuracy: 0.7973

Epoch 00002: val_loss improved from 0.81108 to 0.61714, saving model to best_model_3class.hdf5
Epoch 3/30
150/150 [==============================] - 65s 433ms/step - loss: 0.6302 - accuracy: 0.7828 - val_loss: 0.4980 - val_accuracy: 0.8429

Epoch 00003: val_loss improved from 0.61714 to 0.49803, saving model to best_model_3class.hdf5
Epoch 4/30
150/150 [==============================] - 65s 434ms/step - loss: 0.5057 - accuracy: 0.8249 - val_loss: 0.4307 - val_accuracy: 0.8530

Epoch 00004: val_loss improved from 0.49803 to 0.43072, saving model to best_model_3class.hdf5
Epoch 5/30
150/150 [====================

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

baseline_model = Sequential()

baseline_model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='valid', input_shape=(299, 299, 3)))
baseline_model.add(MaxPooling2D(pool_size=(2, 2)))

baseline_model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', padding='valid'))
baseline_model.add(MaxPooling2D(pool_size=(2, 2)))

# MLP - input layer:
baseline_model.add(Flatten())
# MLP - hidden layer:
baseline_model.add(Dense(128, activation='relu'))
# MLP - output layer:
baseline_model.add(Dense(101, activation='softmax'))

# Check built model:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 170528)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               21827712  
_________________________________________________________________
dense_1 (Dense)              (None, 101)               1

In [ ]:
baseline_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# Common attributes:
steps_per_epoch = 10
epochs = 1000
validation_steps = 10
patience=30
shuffle=True

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator=train_datagen.flow_from_directory(
                   
                    directory="/content/output/train",
                    batch_size=100,
                    seed=55551,
                    shuffle=True,
                    
                    class_mode="categorical",
                    target_size=(299,299))

valid_generator=train_datagen.flow_from_directory(
                    
                    directory="/content/output/val",
                    batch_size=100,
                    seed=55551,
                    shuffle=True,
                  
                    class_mode="categorical",
                    target_size=(299,299))

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(patience=patience, monitor='val_loss', mode='min', restore_best_weights=True)

history_baseline_model = baseline_model.fit_generator(train_generator,
                                                      steps_per_epoch=steps_per_epoch,
                                                      validation_data=valid_generator,
                                                      validation_steps=validation_steps,
                                                      epochs=epochs,
                                                      callbacks=[early_stop],
                                                      shuffle=shuffle
                                                     ) 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
10/10 [==============================] - 23s 2s/step - loss: 8.0918 - acc: 0.0106 - val_loss: 4.6579 - val_acc: 0.0170
Epoch 2/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6411 - acc: 0.0019 - val_loss: 4.6135 - val_acc: 0.0140
Epoch 3/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6148 - acc: 0.0167 - val_loss: 4.6103 - val_acc: 0.0120
Epoch 4/1000
10/10 [==============================] - 14s 1s/step - loss: 4.6083 - acc: 0.0298 - val_loss: 4.6050 - val_acc: 0.0150
Epoch 5/1000
10/10 [==============================] - 13s 1s/step - loss: 4.6106 - acc: 0.0118 - val_loss: 4.6015 - val_acc: 0.0180
Epoch 6/1000
10/10 [==============================] - 13s 1s/step - loss: 4.5967 - acc: 0.0138 - val_loss: 4.5945 - val_acc: 0.0160
Epoch 7/1000
10/10 [==============================] - 13s 1s/step - loss: 4.5939 - acc: 0.0145 - val_loss: 4.5628 - val_acc: 0.0170
Epoch 8/1000
10/10 [==============================] - 13s 1s/step - loss: 4.

# New Section

# New Section

# New Section

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(299, 299, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(101, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 299, 299, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 32)      4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 74, 74, 64)       

In [ ]:
kaggle_baseline_model = model.fit_generator(train_generator,
                                                      steps_per_epoch=steps_per_epoch,
                                                      validation_data=valid_generator,
                                                      validation_steps=validation_steps,
                                                      epochs=epochs,
                                                      callbacks=[early_stop],
                                                      shuffle=shuffle
                                                     )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
10/10 [==============================] - 13s 1s/step - loss: 6.7206 - accuracy: 0.0164 - val_loss: 4.6153 - val_accuracy: 0.0070
Epoch 2/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6151 - accuracy: 0.0130 - val_loss: 4.6152 - val_accuracy: 0.0130
Epoch 3/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6161 - accuracy: 0.0052 - val_loss: 4.6149 - val_accuracy: 0.0130
Epoch 4/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6149 - accuracy: 0.0120 - val_loss: 4.6146 - val_accuracy: 0.0130
Epoch 5/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6154 - accuracy: 0.0131 - val_loss: 4.6148 - val_accuracy: 0.0070
Epoch 6/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6157 - accuracy: 0.0119 - val_loss: 4.6141 - val_accuracy: 0.0130
Epoch 7/1000
10/10 [==============================] - 11s 1s/step - loss: 4.6151 - accuracy: 0.0107 - val_loss: 4.6152 - val_accuracy: 0.0140
Epoch 